# CNN Visualization: Understanding Deep Neural Networks

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Visualize CNN filters and weights** at different layers
2. **Apply Zeiler & Fergus approach** to understand what CNNs learn
3. **Apply Mahendran & Vedaldi approach** for feature inversion
4. **Analyze layer-wise representations** in pre-trained networks
5. **Understand the hierarchical feature learning** in deep networks

## Introduction

Deep neural networks, particularly CNNs, are often considered "black boxes" due to their complexity. This notebook explores various techniques to visualize and understand what these networks learn at different layers.

### Key Visualization Approaches:

1. **Filter Visualization**: Direct visualization of learned weights
2. **Feature Maps**: Visualizing activations at different layers
3. **Feature Inversion**: Reconstructing inputs that maximize specific activations
4. **Gradient-based Methods**: Understanding feature importance

### The VGG16 Architecture

We'll use VGG16, a well-known CNN architecture that won the ImageNet challenge in 2014. It consists of 16 layers with learnable parameters and has a very uniform architecture that makes it ideal for visualization.

## 1. Setup and Imports

In [ ]:
# Install required packages!pip install numpy pandas matplotlib seaborn torch torchvision tqdm scikit-learn pillow# Enable ipywidgets for Jupyter (for interactive dashboard)!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import SGD

from torchvision import models, transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import os

# Import our utility functions
import cnn_utils

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configure matplotlib
plt.style.use('default')
%matplotlib inline

## 2. Load Pre-trained VGG16 Model

In [ ]:
# Load pre-trained VGG16 model
model = models.vgg16(pretrained=True)
model.to(device)
model.eval()  # Set to evaluation mode

print("VGG16 Feature Layers:")
print(model.features)
print("\nVGG16 Classifier:")
print(model.classifier)

In [ ]:
# Extract feature layers for easier access
modules = list(model.features.modules())[1:]  # Remove the Sequential wrapper

print(f"Total number of feature layers: {len(modules)}")
print("\nFirst few layers:")
for i, module in enumerate(modules[:5]):
    print(f"Layer {i}: {module}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {total_params:,}")

## 3. Load and Preprocess Images

In [ ]:
def download_sample_image():
    """Download a sample image for visualization"""
    url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"
    
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img.save('sample_cat.jpg')
        return img
    except:
        print("Could not download image. Please provide your own image.")
        return None

def normalize_image(image):
    """Normalize image for VGG16 input"""
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
    ])
    return preprocess(image).unsqueeze(0).to(device)

# Download and load sample image
img_raw = download_sample_image()
if img_raw:
    plt.figure(figsize=(8, 6))
    plt.imshow(img_raw)
    plt.title("Original Image", fontsize=14)
    plt.axis('off')
    plt.show()
    
    # Preprocess for model input
    img_tensor = normalize_image(img_raw)
    print(f"Input tensor shape: {img_tensor.shape}")
else:
    print("Please add your own image file and update the path below")
    # img_raw = Image.open('your_image.jpg')
    # img_tensor = normalize_image(img_raw)

## 4. Model Prediction

In [ ]:
# Get model prediction
if 'img_tensor' in locals():
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = F.softmax(outputs, dim=1)
        top5_prob, top5_indices = torch.topk(probabilities, 5)
    
    # Load ImageNet class labels
    try:
        # Download ImageNet class labels
        import urllib.request
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt",
            "imagenet_classes.txt"
        )
        
        with open("imagenet_classes.txt", "r") as f:
            classes = [line.strip() for line in f.readlines()]
        
        print("Top 5 Predictions:")
        for i in range(5):
            class_idx = top5_indices[0][i].item()
            prob = top5_prob[0][i].item()
            print(f"{i+1}. {classes[class_idx]}: {prob:.4f}")
    except:
        print("Could not load class labels. Showing raw predictions:")
        print(f"Top prediction index: {top5_indices[0][0].item()}")
        print(f"Confidence: {top5_prob[0][0].item():.4f}")

## 5. Visualize Filter Weights

Let's start by visualizing the actual learned weights in the convolutional filters. These filters are what the network uses to detect features.

In [ ]:
def visualize_conv_weights(model, layer_idx=0, max_filters=64):
    """Visualize convolutional filter weights"""
    
    # Collect all convolutional layers
    conv_layers = []
    for module in model.features.children():
        if isinstance(module, nn.Conv2d):
            conv_layers.append(module)
    
    if layer_idx >= len(conv_layers):
        print(f"Layer index {layer_idx} is out of range. Max index: {len(conv_layers)-1}")
        return
    
    # Get weights from specified layer
    layer = conv_layers[layer_idx]
    weights = layer.weight.data.cpu()
    
    print(f"Layer {layer_idx} weights shape: {weights.shape}")
    print(f"(num_filters={weights.shape[0]}, input_channels={weights.shape[1]}, height={weights.shape[2]}, width={weights.shape[3]})")
    
    # Limit number of filters to visualize
    num_filters = min(max_filters, weights.shape[0])
    grid_size = int(np.ceil(np.sqrt(num_filters)))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(12, 12))
    fig.suptitle(f'Conv Layer {layer_idx} Filter Weights\n{layer}', fontsize=14)
    
    for i in range(grid_size * grid_size):
        row, col = i // grid_size, i % grid_size
        
        if i < num_filters:
            # Average across input channels for visualization
            filter_img = weights[i].mean(dim=0)
            
            # Normalize for better visualization
            filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min())
            
            axes[row, col].imshow(filter_img, cmap='viridis')
            axes[row, col].set_title(f'Filter {i}', fontsize=8)
        
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize first layer filters
visualize_conv_weights(model, layer_idx=0, max_filters=64)

## 6. Visualize Feature Maps at Different Layers

Now let's see how the input image is transformed as it passes through different layers of the network.

In [ ]:
def get_layer_outputs(model, input_tensor):
    """Get outputs from all layers in the feature extractor"""
    outputs = []
    layer_names = []
    
    x = input_tensor
    for i, layer in enumerate(model.features):
        x = layer(x)
        outputs.append(x.clone())
        layer_names.append(f"Layer {i}: {layer.__class__.__name__}")
    
    return outputs, layer_names

def visualize_layer_progression(model, input_tensor):
    """Visualize how image changes through network layers"""
    
    outputs, layer_names = get_layer_outputs(model, input_tensor)
    
    # Select subset of layers to visualize (skip some for clarity)
    indices_to_show = [0, 2, 5, 7, 10, 12, 15, 17, 20, 22, 25, 27, 29, 30]  # Key layers
    
    fig, axes = plt.subplots(4, 4, figsize=(16, 16))
    fig.suptitle('Feature Maps Through Network Layers', fontsize=16)
    
    for idx, layer_idx in enumerate(indices_to_show):
        if idx >= 16 or layer_idx >= len(outputs):
            break
            
        row, col = idx // 4, idx % 4
        
        # Get output and convert to grayscale by averaging across channels
        output = outputs[layer_idx][0].cpu().detach()  # Remove batch dimension
        
        if len(output.shape) == 3:  # Has channels
            # Average across channels
            grayscale = output.mean(dim=0)
        else:
            grayscale = output
        
        axes[row, col].imshow(grayscale, cmap='viridis')
        axes[row, col].set_title(layer_names[layer_idx], fontsize=10)
        axes[row, col].axis('off')
    
    # Hide unused subplots
    for idx in range(len(indices_to_show), 16):
        row, col = idx // 4, idx % 4
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

if 'img_tensor' in locals():
    visualize_layer_progression(model, img_tensor)

## 7. Visualize Individual Filter Responses

Let's examine how different filters in a specific layer respond to our input image.

In [ ]:
def visualize_filter_responses(model, input_tensor, layer_idx, max_filters=64):
    """Visualize individual filter responses at a specific layer"""
    
    # Get output at specified layer
    x = input_tensor
    for i, layer in enumerate(model.features):
        x = layer(x)
        if i == layer_idx:
            target_output = x
            break
    
    # Get filter responses
    filters = target_output[0].cpu().detach()  # Remove batch dimension
    num_filters = min(max_filters, filters.shape[0])
    
    grid_size = int(np.ceil(np.sqrt(num_filters)))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    fig.suptitle(f'Layer {layer_idx} Filter Responses ({filters.shape[0]} total filters)', fontsize=16)
    
    for i in range(grid_size * grid_size):
        row, col = i // grid_size, i % grid_size
        
        if i < num_filters:
            filter_response = filters[i]
            axes[row, col].imshow(filter_response, cmap='viridis')
            axes[row, col].set_title(f'Filter {i}', fontsize=8)
        
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Layer {layer_idx} output shape: {target_output.shape}")
    print(f"Showing first {num_filters} out of {filters.shape[0]} filters")

# Visualize responses from different layers
if 'img_tensor' in locals():
    # First conv layer (low-level features)
    print("=== First Convolutional Layer (Low-level features) ===")
    visualize_filter_responses(model, img_tensor, layer_idx=0, max_filters=64)
    
    # Middle conv layer (mid-level features)
    print("\n=== Middle Convolutional Layer (Mid-level features) ===")
    visualize_filter_responses(model, img_tensor, layer_idx=10, max_filters=64)

## 8. Feature Inversion: Mahendran & Vedaldi Approach

This technique reconstructs an image that produces similar activations to our input image at a specific layer. This helps us understand what information is preserved at different layers.

In [ ]:
def get_layer_output(model, input_tensor, target_layer):
    """Get output from a specific layer"""
    x = input_tensor
    for i, layer in enumerate(model.features):
        x = layer(x)
        if i == target_layer:
            return x
    return x

def feature_inversion(model, input_tensor, target_layer, num_iterations=200, lr=1e3):
    """Perform feature inversion using gradient descent"""
    
    # Get target representation
    model.eval()
    with torch.no_grad():
        target_features = get_layer_output(model, input_tensor, target_layer)
    
    # Initialize random noise image
    noise_image = torch.randn_like(input_tensor, requires_grad=True, device=device)
    
    # Optimizer
    optimizer = SGD([noise_image], lr=lr, momentum=0.9)
    
    # Regularization parameters
    alpha_reg_lambda = 1e-7
    alpha_reg_alpha = 6
    tv_reg_lambda = 1e-8
    tv_reg_beta = 2
    
    reconstructed_images = []
    losses = []
    
    print(f"Starting feature inversion for layer {target_layer}...")
    
    for i in range(num_iterations):
        optimizer.zero_grad()
        
        # Get current features
        current_features = get_layer_output(model, noise_image, target_layer)
        
        # Feature reconstruction loss
        feature_loss = cnn_utils.euclidian_loss(target_features, current_features)
        
        # Regularization terms
        alpha_reg = alpha_reg_lambda * cnn_utils.alpha_norm(noise_image, alpha_reg_alpha)
        tv_reg = tv_reg_lambda * cnn_utils.total_variation_norm(noise_image, tv_reg_beta)
        
        # Total loss
        total_loss = feature_loss + alpha_reg + tv_reg
        
        total_loss.backward()
        optimizer.step()
        
        losses.append(total_loss.item())
        
        # Save intermediate results
        if i % (num_iterations // 10) == 0:
            with torch.no_grad():
                img_array = cnn_utils.recreate_image(noise_image)
                reconstructed_images.append(img_array)
            print(f'Iteration {i}: Loss = {total_loss.item():.6f}')
    
    return reconstructed_images, losses

def visualize_inversion_process(reconstructed_images, target_layer):
    """Visualize the feature inversion process"""
    
    num_images = len(reconstructed_images)
    cols = min(5, num_images)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 3*rows))
    fig.suptitle(f'Feature Inversion Process - Layer {target_layer}', fontsize=16)
    
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, img in enumerate(reconstructed_images):
        row, col = i // cols, i % cols
        axes[row, col].imshow(img)
        axes[row, col].set_title(f'Iteration {i * 20}', fontsize=12)
        axes[row, col].axis('off')
    
    # Hide unused subplots
    for i in range(num_images, rows * cols):
        row, col = i // cols, i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Perform feature inversion for different layers
if 'img_tensor' in locals():
    # Low-level features (early layer)
    print("=== Feature Inversion: Early Layer (Low-level features) ===")
    early_reconstructions, early_losses = feature_inversion(model, img_tensor, target_layer=5, num_iterations=100)
    visualize_inversion_process(early_reconstructions, target_layer=5)
    
    # High-level features (later layer)
    print("\n=== Feature Inversion: Later Layer (High-level features) ===")
    late_reconstructions, late_losses = feature_inversion(model, img_tensor, target_layer=18, num_iterations=100)
    visualize_inversion_process(late_reconstructions, target_layer=18)

## 9. Compare Different Layers

Let's compare the final reconstructions from different layers to understand the information preserved at each level.

In [ ]:
def compare_layer_reconstructions(model, input_tensor, layers_to_compare, num_iterations=150):
    """Compare reconstructions from multiple layers"""
    
    reconstructions = {}
    
    for layer in layers_to_compare:
        print(f"Processing layer {layer}...")
        images, losses = feature_inversion(model, input_tensor, layer, num_iterations)
        reconstructions[layer] = images[-1]  # Take final reconstruction
    
    # Visualize comparisons
    fig, axes = plt.subplots(2, len(layers_to_compare) + 1, figsize=(4*(len(layers_to_compare)+1), 8))
    
    # Original image
    original = cnn_utils.recreate_image(input_tensor)
    axes[0, 0].imshow(original)
    axes[0, 0].set_title('Original Image', fontsize=12)
    axes[0, 0].axis('off')
    axes[1, 0].axis('off')
    
    # Reconstructions
    for i, layer in enumerate(layers_to_compare):
        axes[0, i+1].imshow(reconstructions[layer])
        axes[0, i+1].set_title(f'Layer {layer}\nReconstruction', fontsize=12)
        axes[0, i+1].axis('off')
        
        # Show layer type
        layer_info = str(list(model.features)[layer])
        axes[1, i+1].text(0.5, 0.5, layer_info, ha='center', va='center', 
                         fontsize=8, transform=axes[1, i+1].transAxes, wrap=True)
        axes[1, i+1].axis('off')
    
    plt.suptitle('Feature Inversion Comparison Across Layers', fontsize=16)
    plt.tight_layout()
    plt.show()

if 'img_tensor' in locals():
    # Compare different layers
    layers_to_compare = [2, 7, 14, 21, 28]  # Different depth levels
    compare_layer_reconstructions(model, img_tensor, layers_to_compare, num_iterations=100)

## 10. Gradient-based Saliency Maps

Let's create saliency maps to see which parts of the input image are most important for the prediction.

In [ ]:
def generate_saliency_map(model, input_tensor, target_class=None):
    """
    Generate saliency map using gradients
    """
    model.eval()
    input_tensor.requires_grad_()
    
    # Forward pass
    output = model(input_tensor)
    
    if target_class is None:
        # Use the predicted class
        target_class = output.argmax(dim=1)
    
    # Backward pass
    model.zero_grad()
    output[0, target_class].backward()
    
    # Get gradients
    gradients = input_tensor.grad.data.abs()
    
    # Take maximum across color channels
    saliency_map = gradients.max(dim=1)[0]
    
    return saliency_map[0].cpu().numpy()

def visualize_saliency(model, input_tensor, original_image):
    """Visualize saliency map"""
    
    saliency = generate_saliency_map(model, input_tensor)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(original_image)
    axes[0].set_title('Original Image', fontsize=14)
    axes[0].axis('off')
    
    # Saliency map
    im1 = axes[1].imshow(saliency, cmap='hot')
    axes[1].set_title('Saliency Map', fontsize=14)
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Overlay
    axes[2].imshow(original_image)
    axes[2].imshow(saliency, cmap='hot', alpha=0.4)
    axes[2].set_title('Overlay', fontsize=14)
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

if 'img_tensor' in locals() and 'img_raw' in locals():
    print("=== Saliency Map Analysis ===")
    visualize_saliency(model, img_tensor.clone(), img_raw)

---

#Challenge Section

Now it's your turn! Complete these challenges to deepen your understanding of CNN visualization techniques.

## Challenge 1: Multi-Image Analysis

Compare how different types of images (animals, objects, scenes) activate different filters in the same layer.

**Tasks:**
1. Load 3-4 different images from different categories
2. Visualize filter responses for the same layer across all images
3. Identify which filters are consistently active vs. image-specific
4. Analyze the patterns you observe

In [ ]:
# TODO: Implement multi-image analysis
def analyze_multiple_images(model, image_paths, layer_idx):
    """
    Analyze filter responses across multiple images
    """
    # Your implementation here
    pass

# Test with different image categories
# image_paths = ['cat.jpg', 'car.jpg', 'building.jpg', 'flower.jpg']
# analyze_multiple_images(model, image_paths, layer_idx=10)

## Challenge 2: Filter Specialization Analysis

Investigate what specific features individual filters have learned to detect.

**Tasks:**
1. For a specific layer, identify the top 10 most active filters for your input image
2. Create synthetic images that maximally activate each of these filters
3. Analyze what patterns these filters detect
4. Compare early vs. late layer filter specializations

In [ ]:
# TODO: Implement filter specialization analysis
def analyze_filter_specialization(model, input_tensor, layer_idx, top_k=10):
    """
    Analyze what specific filters have learned
    """
    # Your implementation here
    # Hint: Use feature inversion to create images that maximally activate specific filters
    pass

def maximize_filter_activation(model, layer_idx, filter_idx, num_iterations=200):
    """
    Create an image that maximally activates a specific filter
    """
    # Your implementation here
    pass

## Challenge 3: Layer-wise Information Analysis

Quantify how much information about the original image is preserved at different layers.

**Tasks:**
1. Perform feature inversion for layers 1, 5, 10, 15, 20, 25, 30
2. Calculate similarity metrics between original and reconstructed images
3. Plot information preservation vs. layer depth
4. Identify the "information bottleneck" in the network

In [ ]:
# TODO: Implement information preservation analysis
def calculate_image_similarity(img1, img2):
    """
    Calculate similarity between two images using multiple metrics
    """
    # Implement SSIM, PSNR, or other similarity metrics
    pass

def analyze_information_preservation(model, input_tensor, layers_to_test):
    """
    Analyze how much information is preserved at different layers
    """
    # Your implementation here
    pass

# layers_to_test = [1, 5, 10, 15, 20, 25, 30]
# analyze_information_preservation(model, img_tensor, layers_to_test)

## Challenge 4: Adversarial Visualization

Create visualizations that highlight the network's vulnerabilities.

**Tasks:**
1. Generate adversarial examples that fool the network
2. Visualize the difference between original and adversarial images
3. Show how saliency maps change for adversarial examples
4. Analyze which layers are most affected by adversarial perturbations

In [ ]:
# TODO: Implement adversarial visualization
def generate_adversarial_example(model, input_tensor, target_class, epsilon=0.1):
    """
    Generate adversarial example using FGSM attack
    """
    # Your implementation here
    pass

def visualize_adversarial_effects(model, original_tensor, adversarial_tensor):
    """
    Visualize how adversarial examples affect network activations
    """
    # Your implementation here
    pass

## Challenge 5: Custom Architecture Visualization

Apply these visualization techniques to a different architecture.

**Tasks:**
1. Load a different pre-trained model (ResNet, DenseNet, etc.)
2. Adapt the visualization functions for the new architecture
3. Compare the learned representations between VGG16 and your chosen model
4. Analyze architectural differences in terms of feature learning

In [ ]:
# TODO: Implement custom architecture visualization
def load_alternative_model(model_name='resnet50'):
    """
    Load and prepare an alternative architecture for visualization
    """
    # Your implementation here
    pass

def compare_architectures(model1, model2, input_tensor):
    """
    Compare learned representations between different architectures
    """
    # Your implementation here
    pass

## Challenge 6: Interactive Visualization Dashboard

Create an interactive tool for exploring CNN visualizations.

**Tasks:**
1. Create functions that allow users to:
   - Select different layers to visualize
   - Choose specific filters to analyze
   - Upload custom images for analysis
2. Implement real-time feature inversion
3. Add comparison tools for multiple images/layers

In [ ]:
# TODO: Implement interactive visualization dashboard
def create_interactive_dashboard():
    """
    Create an interactive dashboard for CNN visualization
    """
    # You can use ipywidgets for interactive elements
    # from ipywidgets import interact, IntSlider, Dropdown
    pass

## Challenge 7: Analysis Questions

Answer these questions based on your experiments:

1. **Hierarchical Learning**: How do the features learned by early layers differ from those learned by later layers?

2. **Information Flow**: At which layer does the network lose the most spatial information? Why might this be beneficial?

3. **Filter Redundancy**: Do you observe redundant filters? How might this affect network efficiency?

4. **Feature Inversion Quality**: Why do reconstructions from deeper layers look more abstract?

5. **Practical Applications**: How could these visualization techniques be used in real-world applications?

### Your Answers:

**1. Hierarchical Learning:**
<!-- Your answer here -->

**2. Information Flow:**
<!-- Your answer here -->

**3. Filter Redundancy:**
<!-- Your answer here -->

**4. Feature Inversion Quality:**
<!-- Your answer here -->

**5. Practical Applications:**
<!-- Your answer here -->

## Bonus Challenge: Research Paper Implementation

Implement a more advanced visualization technique from recent research:

**Options:**
1. **Grad-CAM**: Implement Gradient-weighted Class Activation Mapping
2. **Integrated Gradients**: Implement the attribution method
3. **LIME**: Implement Local Interpretable Model-agnostic Explanations
4. **Deep Dream**: Implement the Google Deep Dream algorithm

Choose one and implement it from scratch, then compare it with the basic methods we've used.

In [ ]:
# TODO: Implement advanced visualization technique
def implement_gradcam(model, input_tensor, target_layer, target_class):
    """
    Implement Grad-CAM visualization
    """
    # Your implementation here
    pass

def implement_integrated_gradients(model, input_tensor, target_class, steps=50):
    """
    Implement Integrated Gradients
    """
    # Your implementation here
    pass